In [1400]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
import fitz 
import fillpdf 
from fillpdf import fillpdfs

In [1401]:
#############################################################################################################
#####################################  RFL Volume Simulation 2024-2029  #####################################
#############################################################################################################

# Constants
GLOBAL_PIFs = 250000
INITIAL_POPULATION = GLOBAL_PIFs*.55
INITIAL_TARGET_2024 = 4000
INITIAL_TARGET_2025 = 50000
TARGET_INCREASE_RATE = 0.10
POPULATION_GROWTH_RATE = 0.08
LIGHT_RENEWAL_RATE = 0.98
FULL_RENEWAL_RATE = 0.98
YEARLY_LIGHT_BIAS_DECREASE = 0.1
LIGHT_RIDER_PREMIUM = 0.1
light_bias = 0.7

# Conversion rates
FULL_CONVERSION_RATE = 0.36
LIGHT_CONVERSION_MULTIPLIER = 1.2
LIGHT_CONVERSION_RATE = FULL_CONVERSION_RATE * LIGHT_CONVERSION_MULTIPLIER
LIGHT_TO_FULL_UPGRADE_RATE = 0.8  ## Year 2, Rider customers will upgrade to Full at ~80% conversion rate.

# Revenue bands distribution
REVENUE_BANDS = {
    "<20M": 0.8,
    "20-50M": 0.1,
    "50-100M": 0.06,
    "100M-4B": 0.03,
    ">4B": 0.01
}

# Initialize population distribution by revenue bands
population_by_band = {band: INITIAL_POPULATION * percentage for band, percentage in REVENUE_BANDS.items()}

# Function to simulate one year
def simulate_year(year, available_population, target_customers, light_bias, retained_light_customers_prev, retained_full_customers_prev, new_light_customers_prev, new_full_customers_prev):
    new_light_customers = {}
    new_full_customers = {}
    retained_light_customers = {}
    retained_full_customers = {}
    next_available_population = {}

    for band in REVENUE_BANDS.keys():
        available_band_population = available_population[band]
        target_band_customers = target_customers * REVENUE_BANDS[band]

        # Calculate new customers
        light_customers = int(target_band_customers * light_bias * LIGHT_CONVERSION_RATE)
        full_customers = int(target_band_customers * (1 - light_bias) * FULL_CONVERSION_RATE)

        # Calculate retained customers from previous year
        retained_light = int((retained_light_customers_prev[band] + new_light_customers_prev[band]) * LIGHT_RENEWAL_RATE)
        retained_full = int((retained_full_customers_prev[band] + new_full_customers_prev[band]) * FULL_RENEWAL_RATE)

        new_light_customers[band] = light_customers
        new_full_customers[band] = full_customers
        retained_light_customers[band] = retained_light
        retained_full_customers[band] = retained_full

        # Calculate next year's available population
        next_available_population[band] = int((available_band_population - light_customers - full_customers) * (1 + POPULATION_GROWTH_RATE))

    return new_light_customers, retained_light_customers, new_full_customers, retained_full_customers, next_available_population

# Initialize data structures for simulation
data = []

# Simulate each year
available_population = population_by_band.copy()
target_customers = INITIAL_TARGET_2024

retained_light_customers_prev = {band: 0 for band in REVENUE_BANDS.keys()}
retained_full_customers_prev = {band: 0 for band in REVENUE_BANDS.keys()}
new_light_customers_prev = {band: 0 for band in REVENUE_BANDS.keys()}
new_full_customers_prev = {band: 0 for band in REVENUE_BANDS.keys()}

for year in range(2024, 2030):
    if year == 2025:
        target_customers = INITIAL_TARGET_2025
    
    new_light_customers, retained_light_customers, new_full_customers, retained_full_customers, next_available_population = simulate_year(
        year, available_population, target_customers, light_bias, retained_light_customers_prev, retained_full_customers_prev, new_light_customers_prev, new_full_customers_prev
    )

    for band in REVENUE_BANDS.keys():
        data.append([
            year,
            band,
            available_population[band],
            new_light_customers[band],
            retained_light_customers[band],
            new_full_customers[band],
            retained_full_customers[band]
        ])

    # Update for next year
    available_population = next_available_population
    target_customers = int(target_customers * (1 + TARGET_INCREASE_RATE))
    light_bias = max(0, light_bias - YEARLY_LIGHT_BIAS_DECREASE)
    retained_light_customers_prev = retained_light_customers.copy()
    retained_full_customers_prev = retained_full_customers.copy()
    new_light_customers_prev = new_light_customers.copy()
    new_full_customers_prev = new_full_customers.copy()

# Create DataFrame
columns = ["Year", "Revenue Band", "Originating Population", "New Light Customer Count", "Retained Light Customer Count", "New Full Customer Count", "Retained Full Customer Count"]
df = pd.DataFrame(data, columns=columns)
df['Total Light Customer Count'] = df['New Light Customer Count'] + df['Retained Light Customer Count']
df['Total Full Customer Count'] = df['New Full Customer Count'] + df['Retained Full Customer Count']


In [1402]:
#############################################################################################################
###################################  Cyber Premium Index Reference Table  ###################################
#############################################################################################################

# Read the DataFrame from the CSV file
gwpIndex = pd.read_csv('gwpIndex.csv')

# Convert companyRevenue column to numeric, forcing errors to NaN
gwpIndex['companyRevenue'] = pd.to_numeric(gwpIndex['companyRevenue'], errors='coerce')

# Function to determine the 'band' based on 'companyRevenue'
def determine_band(revenue):
    if revenue <= 20000000:
        return '<20M'
    elif 20000001 <= revenue <= 50000000:
        return '20-50M'
    elif 50000001 <= revenue <= 100000000:
        return '50-100M'
    elif 100000001 <= revenue <= 4000000000:
        return '100M-4B'
    else:
        return '>4B'

# Apply the function to create the 'band' column
gwpIndex['Revenue Band'] = gwpIndex['companyRevenue'].apply(determine_band)

# Group by 'band' and calculate the average of 'CyberBWP'
gwpIndexAvg = gwpIndex.groupby('Revenue Band')['Cyber GWP'].mean().reset_index()

# Replace values based on 'Revenue Band'
gwpIndexAvg.loc[gwpIndexAvg['Revenue Band'] == '100M-4B', 'Cyber GWP'] = 55736.01
gwpIndexAvg.loc[gwpIndexAvg['Revenue Band'] == '50-100M', 'Cyber GWP'] = 15319.15
gwpIndexAvg.loc[gwpIndexAvg['Revenue Band'] == '20-50M', 'Cyber GWP'] = 16863.96
gwpIndexAvg.loc[gwpIndexAvg['Revenue Band'] == '<20M', 'Cyber GWP'] = 4267.64

In [1407]:
#############################################################################################################
##########################################  Calculate GWPs by Year  #########################################
#############################################################################################################

# Merge 'gwpIndexAvg' with 'df' using 'Revenue Band' as the key
merged_df = pd.merge(df, gwpIndexAvg, on='Revenue Band', how='left')
merged_df.rename(columns={'Cyber GWP': 'Full Cyber Premium'}, inplace=True)
merged_df['Rider Cyber Premium'] = merged_df['Full Cyber Premium'] * LIGHT_RIDER_PREMIUM
merged_df['Full Cyber GWP'] = merged_df['Full Cyber Premium'] *  merged_df['Total Full Customer Count']
merged_df['Rider Cyber GWP'] = merged_df['Full Cyber Premium'] *  merged_df['Total Light Customer Count']
merged_df['Combined GWP'] = merged_df['Full Cyber GWP'] + merged_df['Rider Cyber GWP']

,Year,Revenue Band,Originating Population,New Light Customer Count,Retained Light Customer Count,New Full Customer Count,Retained Full Customer Count,Total Light Customer Count,Total Full Customer Count,Full Cyber Premium,Rider Cyber Premium,Full Cyber GWP,Rider Cyber GWP,Combined GWP
0,2024,<20M,110000.0,967,0,345,0,967,345,4267.64,426.764,1.472336e+06,4.126808e+06,5.599144e+06
1,2024,20-50M,13750.0,120,0,43,0,120,43,16863.96,1686.396,7.251503e+05,2.023675e+06,2.748825e+06
2,2024,50-100M,8250.0,72,0,25,0,72,25,15319.15,1531.915,3.829788e+05,1.102979e+06,1.485958e+06
3,2024,100M-4B,4125.0,36,0,12,0,36,12,55736.01,5573.601,6.688321e+05,2.006496e+06,2.675328e+06
4,2024,>4B,1375.0,12,0,4,0,12,4,1250.00,125.000,5.000000e+03,1.500000e+04,2.000000e+04
5,2025,<20M,117383.0,10368,947,5760,338,11315,6098,4267.64,426.764,2.602407e+07,4.828835e+07,7.431242e+07
6,2025,20-50M,14673.0,1296,117,720,42,1413,762,16863.96,1686.396,1.285034e+07,2.382878e+07,3.667911e+07
7,2025,50-100M,8805.0,777,70,432,24,847,456,15319.15,1531.915,6.985532e+06,1.297532e+07,1.996085e+07
8,2025,100M-4B,4403.0,388,35,216,11,423,227,55736.01,5573.601,1.265207e+07,2.357633e+07,3.622841e+07
9,2025,>4B,1467.0,129,11,72,3,140,75,1250.00,125.000,9.375000e+04,1.750000e+05,2.687500e+05


In [1404]:
#assumptions
Max_Cost_Dist = .22
Ryan_MGU_Share = .1 
Ceded_Premiums = .13
Loss_Ratio = .43
Operating_exp = .1
Margin_safety = .3

data = {}
years = [2024, 2025, 2026, 2027, 2028, 2029]
formatting = pd.read_csv('format.csv')

def yearly_sum(year, merged_df):
    #dataframe of only the year we're adding 
    filtered_df = merged_df.loc[merged_df['Year'] == year]

    sum_list = []
    for col_name in list(merged_df.columns[2:7]) + list(merged_df.columns[11:]):
        sum = 0
    
        for i in range(5):
            sum += filtered_df.iloc[i,filtered_df.columns.get_loc(col_name)]
        sum_list.append(round(sum,2))

    rows_0_to_5 = formatting.iloc[0, 0:5].values.tolist()  # Extract values from row 0, columns 0 to 4
    rows_8_and_9 = formatting.iloc[0, 7:9].values.tolist()  # Extract values from row 0, columns 8 to 9

    second_row = rows_0_to_5 + rows_8_and_9


    #dict with eacher year as a key and a dict of the sums and their names on the proforma
    data[year] = dict(zip(second_row, sum_list))

    #update with switched values
    var = data[year]["New Full Customers"] 
    data[year]["New Full Customers"] = data[year]["Retained Rider Customers"]
    data[year]["Retained Rider Customers"] = var

    var1 = (data[year]["Rider Cyber GWP (gross written premiums)"] )
    (data[year]["Rider Cyber GWP (gross written premiums)"] ) = data[year]["Full Cyber GWP (gross written premiums)"]
    data[year]["Full Cyber GWP (gross written premiums)"] = var1




def yearly_assumptions(data, year):
    mult_list = []
    second_row = list(formatting.iloc[0, 5:7]) + list(formatting.iloc[0, 9:])

    Comb_GWP = (data[year]["Rider Cyber GWP (gross written premiums)"] + data[year]["Full Cyber GWP (gross written premiums)"])

    Total_Rider_PIF = data[year]["New Rider Customers"] + data[year]["Retained Rider Customers"]
    Total_Full_PIF = data[year]["New Full Customers"] + data[year]["Retained Full Customers"]
    Comb_GWP_Applied = Comb_GWP * (1 - Margin_safety)
    Ceded_Reinsurance = Comb_GWP_Applied * (Ceded_Premiums)
    Dist_cost = Comb_GWP_Applied * (Max_Cost_Dist)
    Ryan_Share = Comb_GWP_Applied * ( Ryan_MGU_Share)
    Total_COGS = Ceded_Reinsurance + Dist_cost + Ryan_Share
    Loss_Reserves = Comb_GWP_Applied * (Loss_Ratio)
    General_Admin = Comb_GWP_Applied * (Operating_exp)
    Total_Operating = Loss_Reserves + General_Admin
    Underwriting_Profit = Comb_GWP_Applied - (Total_COGS + Total_Operating)
    Underwriting_Profit_Margin = (Underwriting_Profit / Comb_GWP_Applied)

    # Append calculated values to the list
    mult_list.extend([
        Total_Rider_PIF,
        Total_Full_PIF,
        Comb_GWP,
        Comb_GWP_Applied,
        Ceded_Reinsurance,
        Dist_cost,
        Ryan_Share,
        Total_COGS,
        Loss_Reserves,
        General_Admin,
        Total_Operating,
        Underwriting_Profit,
        (Underwriting_Profit_Margin *100)
    ])

    data[year].update(dict(zip(second_row, mult_list)))



for year in years:
    yearly_sum(year, merged_df)
    yearly_assumptions(data, year)

data = pd.DataFrame(data)

In [1405]:
# Open the existing PDF
pdf_path = 'RFL_proForma_Template.pdf'
output_path = 'ProFormaPython.pdf'

pdf_document = fitz.open(pdf_path)
page = pdf_document[0]


def add_left_text(page, point, text, fontname="times-roman", fontsize=7.5):
    # Create a font object
    text = text.replace(',', ', ')

    font = fitz.Font(fontname)
    
    # Get the length of the text
    text_length = font.text_length(text, fontsize=fontsize)
    
    # Calculate the center position
    x_center, y_center = point
    x_start = x_center
    y_start = y_center - (fontsize / 2)
    
    # Insert the text centered around the point
    page.insert_text((x_start, y_start), text, fontname=fontname, fontsize=fontsize)

X_grid = {
    2024: 263,
    2025: 353.5,
    2026: 423,
    2027: 493,
    2028: 563,
    2029: 634
}

Y_grid = [
    205.25, # universe
    228,    # new rider
    241,    # new full
    262.25,    # retained rider
    274.25,    # retained full
    294.5,    # total rider pif
    307.5,    # total full pif
    333,    # rider cyber
    343,    # full cyber 
    362,    # combined gross written premiums
    381.5,    # margin of safety
    413,    # ceded reinsurance (updated)
    442,    # distributor (updated)
    454,    # ryanmcu (updated)
    471,    # total cogs (updated)
    500,    # loss reserves (updated)
    509,    # general administration (updated)
    529.5,    # total operating (updated)
    549.5,    # total operating continuation (updated)
    561.5     # total operating end (updated)
]




def get_text(data,year, formatting):
    second_row = list(formatting.iloc[0, :])
    yearly_list = []
    
    for item in second_row:
        value = data[year][item]
        if value < 1 and value > 0:
            num = round(float(value), 2)
            yearly_list.append(num)
        elif value == 0:
                yearly_list.append(0)
    
        else:
            yearly_list.append(int(round(value, 2)))  # Convert to integer
    
    
    formatted_list = ['{:,}'.format(num) for num in yearly_list]
    formatted_list.pop(-1)

    formatted_list.append(str(yearly_list[-1]) + "%")
    return formatted_list


def print_col(data,X_grid,Y_grid,year):
    yearly_list = get_text(data,year,formatting)
    for index in range(len(yearly_list)):
        cord = (X_grid[year],Y_grid[index])
        add_left_text(page,cord,yearly_list[index])

def print_assumtions():
    y_value = 74
    offset = 0
    assumptions = [Max_Cost_Dist,Ryan_MGU_Share,Ceded_Premiums,Loss_Ratio,Operating_exp,"",Margin_safety]
    comb_ratio = Max_Cost_Dist+Ryan_MGU_Share+Ceded_Premiums+Loss_Ratio+Operating_exp
    empty_index = assumptions.index("")
    assumptions.insert(empty_index, comb_ratio)

    for i in range(len(assumptions)):
        cord = ((X_grid[2024]-5), y_value + offset)
        
        # Check if the current assumption is an empty string
        if assumptions[i] == "":
            add_left_text(page, cord, "")
        else:
            add_left_text(page, cord, str((assumptions[i]) * 100) + "%")
        
        offset += 10



In [1406]:
for year in years:
    print_col(data,X_grid,Y_grid,year)
print_assumtions()

pdf_document.save(output_path)